# Aufgabe 1
**a) Lesen Sie die CSV-Dateien,die die Stromerzeugungsdaten und Börsenstrompreise enthalten ein und führen Sie sie in einem DataFrame namens `df_hourly` zusammen.**

In [ ]:
import os

import numpy as np
import pandas as pd
path = 'DAT-WS2425-Projektarbeit\DAT-WS2425-Projektarbeit\Daten\Strompreisdaten'
extension = '.csv'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
files = [file for file in os.listdir(path) if file.endswith(extension)]

dfs = []
filecounter =0
for file in files:
    df = pd.read_csv(os.path.join(path, file))
    dfs.append(df)
    filecounter += 1
    
print("total files: ", filecounter)
df = pd.concat(dfs, ignore_index=True)

**b) Passen Sie die dtypes der Spalten von `df_hourly` geeignet an. Überführen Sie insbesondere das Datum in ein DateTime-Format. Entfernen Sie alle Datensätze, die sich nicht auf
 den Betrachtungszeitraum 2020-2024 beziehen.**

In [27]:
df['Datum (MEZ)'] = pd.to_datetime(df['Datum (MEZ)'])
df['Datum (MESZ)'] = pd.to_datetime(df['Datum (MESZ)'])
print(len(df['Datum (MESZ)']))

43392


In [28]:
df = df.drop(df[df['Datum (MEZ)']<'2020-01-01'].index)

**c) Beurteilen Sie die Datenqualität des Datensatzes und führen Sie, sofern aus Ihrer Sicht notwendig, geeignete Datenbereinigungsschritte durch.**

In [29]:
# Separating and Creating new columns for Date and Time from Datum (MEZ) and Datum (MESZ)
df['Datum'] = df['Datum (MEZ)'].dt.date
df['Zeit'] = df['Datum (MEZ)'].dt.time
df['Datum'] =df['Datum'].fillna(df['Datum (MESZ)'].dt.date)
df['Zeit'] =df['Zeit'].fillna(df['Datum (MESZ)'].dt.time)
cols = df.columns.tolist()
print(cols)

['Datum (MEZ)', 'Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)', 'Day Ahead Auktion Preis (EUR/MWh)', 'Datum (MESZ)', 'Datum', 'Zeit']


In [30]:
cols = ['Datum', 'Zeit','Datum (MEZ)', 'Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)', 'Day Ahead Auktion Preis (EUR/MWh)', 'Datum (MESZ)']
df = df[cols]
df = df.drop(columns=['Datum (MEZ)', 'Datum (MESZ)'])
df = df.reset_index(drop=True)
df

,Datum,Zeit,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
0,2020-01-01,00:00:00,18754.70,13961.99,41.88
1,2020-01-01,01:00:00,18337.54,14242.17,38.60
2,2020-01-01,02:00:00,18122.83,14584.20,36.55
3,2020-01-01,03:00:00,18323.93,14746.24,32.32
4,2020-01-01,04:00:00,18327.59,14924.48,30.85
...,...,...,...,...,...
43339,2024-12-10,19:00:00,NaN,NaN,180.17
43340,2024-12-10,20:00:00,NaN,NaN,166.60
43341,2024-12-10,21:00:00,NaN,NaN,151.15
43342,2024-12-10,22:00:00,NaN,NaN,137.94


**d) Erzeugen Sie aus `df_hourly` einen weiteren DataFrame namens `df_daily`, der in jeder
 Zeile die erzeugte elektrische Energie mit erneuerbaren und nicht erneuerbaren Energie
trägern sowie den an diesem Tag durchschnittlich gemessenen Börsenstrompreis enthält.**

In [53]:
# New df_daily with sum of Leistung and average of Auktion preis for every day
df_daily = df.groupby(['Datum']).agg({'Leistung nicht erneuerbar (MW)': 'sum', 'Leistung erneuerbar (MW)': 'sum', 'Day Ahead Auktion Preis (EUR/MWh)' : 'mean'})
df_daily

,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
Datum,,,
2020-01-01,468837.03,441093.57,34.964167
2020-01-02,505002.46,605620.76,38.869167
2020-01-03,427798.33,968577.88,26.111250
2020-01-04,411321.40,987041.46,20.322917
2020-01-05,508187.80,514534.00,35.552500
...,...,...,...
2024-12-06,482833.73,969065.85,86.374583
2024-12-07,386654.54,834428.48,71.508333
2024-12-08,455016.51,609461.18,99.068333


# Aufagbe 2

**a) An welchen 10 Tagen im Betrachtungszeitrum wurde am meisten Strom aus erneuerbaren Energieträgern erzeugt?**

In [32]:
df_daily.sort_values('Leistung erneuerbar (MW)', ascending=False).head(10)

,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
Datum,,,
2024-02-06,331773.90,1380463.41,45.292917
2024-02-05,331039.23,1358926.97,35.840833
2024-01-24,351983.99,1316391.11,33.122917
2023-12-21,275886.68,1308032.94,25.280417
2023-12-22,242307.14,1289984.86,22.655833
2022-04-07,396999.16,1274027.01,74.153333
2022-02-17,381238.33,1271433.30,68.637083
2024-09-27,198813.32,1269933.18,14.757500
2023-12-29,221981.80,1261365.77,6.359583


**b) An welchem Tag im Betrachtungszeitraum wurde der höchste Bösenstrompreis verzeichnet und wie hoch war er? An welchem Tag wurde der geringste Preis verzeichnet und wie hoch war er?**

In [33]:
df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] == df_daily['Day Ahead Auktion Preis (EUR/MWh)'].max()]

,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
Datum,,,
2022-08-26,640735.36,493492.08,699.441667


In [34]:
df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] == df_daily['Day Ahead Auktion Preis (EUR/MWh)'].min()]

,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
Datum,,,
2023-07-02,183483.62,940768.39,-53.870833


**c) Wie viele Tage gab es im Betrachtungszeitraum 2020-2024, an denen ein negativer Börsenstrompreis aufgetreten ist?**

In [39]:
print(len(df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] < 0]))

13


In [40]:
df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] < 0]

,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
Datum,,,
2020-02-16,252739.69,1152357.76,-8.274167
2020-02-22,243166.11,1236952.50,-1.508750
2020-03-22,260187.93,969434.27,-0.947083
2020-04-13,204282.71,875000.49,-15.966250
2020-04-21,218578.73,1151305.34,-16.149583
2020-05-24,205203.09,858451.60,-26.130833
2020-07-05,188708.17,976626.02,-15.337083
2020-12-27,236977.55,1093977.66,-13.452917
2021-04-05,227435.87,1056971.92,-16.996667


**d) Wie viel Strom wurde pro Jahr mit erneuerbaren und mit nicht erneuerbaren Energieträgernerzeugt?**

In [67]:
df2 = df

In [68]:
df2['Jahr'] = pd.DatetimeIndex(df2['Datum']).year
df_yearly = df2.groupby(['Jahr']).agg({'Leistung nicht erneuerbar (MW)':'sum', 'Leistung erneuerbar (MW)':'sum'})
df_yearly

,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW)
Jahr,,
2020,1.870113e+08,2.392726e+08
2021,2.093507e+08,2.220113e+08
2022,2.168424e+08,2.380835e+08
2023,1.653949e+08,2.534251e+08
2024,1.411188e+08,2.441503e+08
